In [ ]:
!pip install -q transformers
!pip install -q evaluate
!pip install -q datasets
!pip install -q torch
!pip install -q torchinfo
!pip install -q tqdm
!pip install -q rouge_score
# !pip install -q pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.

In [ ]:
import pandas as pd
import evaluate
import numpy as np
import torch

from transformers import BertTokenizer, RobertaTokenizer, AutoTokenizer, AutoModel, BertForQuestionAnswering, RobertaForQuestionAnswering, AutoModelForQuestionAnswering, Trainer, TrainingArguments, pipeline
from torchinfo import summary
from datasets import Dataset, load_dataset
from pprint import pprint
from tqdm import tqdm

In [ ]:
ds = load_dataset("rony/climate-change-MRC")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/633 [00:00<?, ?B/s]

CCMRC_train.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

CCMRC_validation.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

CCMRC_test.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
def prepare_data(data):
  articles = []

  for paragraph in data:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      for ans in qa['answers']:
        answer = ans['text']
        answer_start = ans['answer_start']
        articles.append({'context': context, 'question': question, 'id': id, 'answer': answer, 'answer_start': answer_start})

  return articles

In [ ]:
train_ds = ds["train"]
valid_ds = ds["validation"]
test_ds = ds["test"]

# each is a 1-item list, so take first index
train_ds = train_ds[0]
valid_ds = valid_ds[0]
test_ds = test_ds[0]

# take the 'data' key of the dict, ignoring 'version' (there's just one)
train_ds = train_ds['data'][0]['paragraphs']
valid_ds = valid_ds['data'][0]['paragraphs']
test_ds = test_ds['data'][0]['paragraphs']
# each dataset is a list of dicts, where each list item is a context paragraph ('context' key) with qas ('qas' key) which contain questions, id, and answer

train_df = pd.DataFrame(prepare_data(train_ds))
print(f"{train_df.shape=}")

valid_df = pd.DataFrame(prepare_data(valid_ds))
print(f"{valid_df.shape=}")

test_df = pd.DataFrame(prepare_data(test_ds))
print(f"{test_df.shape=}")

train_df.shape=(14756, 5)
valid_df.shape=(4229, 5)
test_df.shape=(2096, 5)


In [ ]:
train_df.head()

,context,question,id,answer,answer_start
0,"outside of the clean air act, there is support...",Is a mandate for electric production to come f...,665,"and there is ongoing interest in a ""national r...",222
1,"outside of the clean air act, there is support...",How wide ranging are climate policies across t...,666,"as of 2010, climate policies were being contem...",618
2,"outside of the clean air act, there is support...",What kind of energy standards have been implem...,667,"at latest count, 30 states have implemented re...",734
3,acknowledgments. we thank jim haywood for the ...,What was Jim Haywood thanked for?,1731,we thank jim haywood for the aerosol forcing pdf,17
4,acknowledgments. we thank jim haywood for the ...,Where was Jonathan Gregory supported?,1732,jonathan gregory was supported at the universi...,277


In [ ]:
# function that tokenizes question and context input data and sends through model to get answers from model output

def get_answer_from_model(context, question):
    """Passes context and question through tokenizer, sends tokenized inputs through model and returns answer"""
    inputs = tokenizer(question, context, truncation=True, return_tensors='pt')

    outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = inputs['input_ids'][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens)

    return answer

In [ ]:
# BERT base

# define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# run on test data
tqdm.pandas()
test_df['bert_answer'] = test_df.progress_apply(
    lambda row: get_answer_from_model(row['context'], row['question']), axis=1
)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  7%|▋         | 146/2096 [02:48<37:28,  1.15s/it]  


KeyboardInterrupt: 

In [ ]:
test_df.to_parquet('test_bert_scored.parquet')
test_df.head()

,context,question,id,answer,answer_start,bert_answer
0,some more detailed work has been done at natio...,The 9 percent reduction of rice in Bangladesh ...,14095,flooding damage and climate variability,514,flooding damage and climate variability
1,some more detailed work has been done at natio...,What kind of model of Bangladesh was had been ...,14096,a dynamic economywide model,70,dynamic economywide model
2,some more detailed work has been done at natio...,What approach did Ahmed use to estimate how ch...,14097,a modelling approach,639,modelling
3,extreme sea level height fluctuations are also...,Where height fluctuations are large?,2843,extreme sea level height fluctuations are also...,0,to the north
4,extreme sea level height fluctuations are also...,How non-tide sea levels are obtained?,2844,the non-tide sea levels are obtained by spectr...,167,by spectrally removing the tidal energy from t...


In [ ]:
# read in BERT base answers
test_df = pd.read_parquet('test_bert_scored.parquet')

# evaluate BERT base answers
rouge = evaluate.load('rouge')
predictions = test_df['bert_answer'].tolist()
references = test_df['answer'].tolist()
results = rouge.compute(predictions=predictions,
                       references=references)
print('test \n',results)

  Preparing metadata (setup.py) ... done
test 
 {'rouge1': np.float64(0.2739855608014806), 'rouge2': np.float64(0.2373774320013533), 'rougeL': np.float64(0.2711455265995975), 'rougeLsum': np.float64(0.27122919944726476)}


In [ ]:
# DistilRoBERTa base

# define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilroberta-base")

# run on test data
tqdm.pandas()
test_df['distilroberta_answer'] = test_df.progress_apply(
    lambda row: get_answer_from_model(row['context'], row['question']), axis=1
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2096/2096 [10:41<00:00,  3.27it/s]


In [ ]:
test_df.to_parquet('test_distilroberta_scored.parquet')
test_df.head()

,context,question,id,answer,answer_start,distilroberta_answer
0,some more detailed work has been done at natio...,The 9 percent reduction of rice in Bangladesh ...,14095,flooding damage and climate variability,514,
1,some more detailed work has been done at natio...,What kind of model of Bangladesh was had been ...,14096,a dynamic economywide model,70,
2,some more detailed work has been done at natio...,What approach did Ahmed use to estimate how ch...,14097,a modelling approach,639,
3,extreme sea level height fluctuations are also...,Where height fluctuations are large?,2843,extreme sea level height fluctuations are also...,0,"al. 2003 ). note that, for example, a 30 cm e..."
4,extreme sea level height fluctuations are also...,How non-tide sea levels are obtained?,2844,the non-tide sea levels are obtained by spectr...,167,"al. 2003 ). note that, for example, a 30 cm e..."


In [ ]:
# read in DistilRoBERTa answers
test_df = pd.read_parquet('test_distilroberta_scored.parquet')

# evaluate DistilRoBERTa answers
rouge = evaluate.load('rouge')
predictions = test_df['distilroberta_answer'].tolist()
references = test_df['answer'].tolist()
results = rouge.compute(predictions=predictions,
                       references=references)
print('test \n',results)

test 
 {'rouge1': np.float64(0.19998494093148167), 'rouge2': np.float64(0.13268701236279767), 'rougeL': np.float64(0.16757016054097706), 'rougeLsum': np.float64(0.16774135132614454)}


In [ ]:
def tokenize_function(examples):

  """Main tokenize function"""

  tokenized_inputs = tokenizer(
      examples["question"],
      examples["context"],
      truncation="only_second",
      padding="max_length",
      max_length=512,
  )

  start_positions = []
  end_positions = []

  for i in range(len(examples["answer"])):
      context = examples["context"][i]
      answer = examples["answer"][i]
      answer_start = examples["answer_start"][i]
      answer_end = answer_start + len(answer) - 1

      start_token = tokenizer.encode(context[:answer_start], add_special_tokens=False)
      end_token = tokenizer.encode(context[:answer_end + 1], add_special_tokens=False)

      start_positions.append(len(start_token))
      end_positions.append(len(end_token) - 1)

  tokenized_inputs["start_positions"] = start_positions
  tokenized_inputs["end_positions"] = end_positions

  return tokenized_inputs

In [ ]:
# DistilRoBERTa base model

## Define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilroberta-base")

## Freeze layers and only train QA head
for param in model.roberta.parameters():
    param.requires_grad = False

for param in model.qa_outputs.parameters():
    param.requires_grad = True

## Tokenize train, validation, and test sets using main tokenize function
roberta_train = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
roberta_valid = Dataset.from_pandas(valid_df).map(tokenize_function, batched=True)
roberta_test = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

## Define the training arguments
roberta_training_args = TrainingArguments(
    output_dir="./distilroberta_results",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

## Prepare the Trainer
roberta_trainer = Trainer(
    model=model,
    args=roberta_training_args,
    train_dataset=roberta_train,
    eval_dataset=roberta_valid
)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/14756 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/4229 [00:00<?, ? examples/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
## Train the DistilRoBERTa base model
roberta_trainer.train()

## Evaluate the DistilRoBERTa base model on the validation set
val_results = roberta_trainer.evaluate()
print(val_results)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mjlenci (mjlenci-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,5.957700,5.599524
2,5.592800,5.502632
3,5.533400,5.481114


{'eval_loss': 5.481114387512207, 'eval_runtime': 59.6868, 'eval_samples_per_second': 70.853, 'eval_steps_per_second': 4.44, 'epoch': 3.0}


In [ ]:
## Compute ROUGE scores for DistilRoBERTa test

roberta_predictions = roberta_trainer.predict(roberta_test)
logits = roberta_predictions.predictions
start_logits = torch.tensor(logits[0])
end_logits = torch.tensor(logits[1])

start_positions = torch.argmax(start_logits, dim=-1)
end_positions = torch.argmax(end_logits, dim=-1)

decoded_answers = []
for i, (start, end) in enumerate(zip(start_positions, end_positions)):
    # context = tokenized_valid[i]['context']

    # Convert token positions back to text
    # answer = context[start:end+1]  # slice the context based on start and end token positions
    # decoded_answers.append(answer)

    answer_tokens = roberta_test[i]['input_ids'][start:end+1]
    decoded_answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    decoded_answers.append(decoded_answer)

references = test_df['answer'].tolist()
rouge_results = rouge.compute(predictions=decoded_answers, references=references)
print(rouge_results)

{'rouge1': np.float64(0.3417676191614125), 'rouge2': np.float64(0.22910381540236047), 'rougeL': np.float64(0.2897479321838424), 'rougeLsum': np.float64(0.29014758601882396)}


In [ ]:
roberta_df = test_df.copy()
roberta_df['finetuned_distilroberta_answer'] = decoded_answers

roberta_df.to_parquet('test_roberta_scored.parquet')
roberta_df.head()

,context,question,id,answer,answer_start,distilroberta_answer,finetuned_distilroberta_answer
0,some more detailed work has been done at natio...,The 9 percent reduction of rice in Bangladesh ...,14095,flooding damage and climate variability,514,,The 9 percent reduction of rice in Bangladesh ...
1,some more detailed work has been done at natio...,What kind of model of Bangladesh was had been ...,14096,a dynamic economywide model,70,,What kind of model of Bangladesh was had been ...
2,some more detailed work has been done at natio...,What approach did Ahmed use to estimate how ch...,14097,a modelling approach,639,,What approach did Ahmed use to estimate how ch...
3,extreme sea level height fluctuations are also...,Where height fluctuations are large?,2843,extreme sea level height fluctuations are also...,0,"al. 2003 ). note that, for example, a 30 cm e...",Where height fluctuations are large?extreme s...
4,extreme sea level height fluctuations are also...,How non-tide sea levels are obtained?,2844,the non-tide sea levels are obtained by spectr...,167,"al. 2003 ). note that, for example, a 30 cm e...",How non-tide sea levels are obtained?extreme ...


In [ ]:
# ClimateBERT model fine tuned for climate domain

# define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")
model = AutoModelForQuestionAnswering.from_pretrained("climatebert/distilroberta-base-climate-f")

# run on test data
test_df['climatebert_answer'] = test_df.progress_apply(
    lambda row: get_answer_from_model(row['context'], row['question']), axis=1
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2096/2096 [12:23<00:00,  2.82it/s]


In [ ]:
test_df.to_parquet('test_climatebert_scored.parquet')
test_df.head()

,context,question,id,answer,answer_start,distilroberta_answer,climatebert_answer
0,some more detailed work has been done at natio...,The 9 percent reduction of rice in Bangladesh ...,14095,flooding damage and climate variability,514,,"century, and most of these losses are attribut..."
1,some more detailed work has been done at natio...,What kind of model of Bangladesh was had been ...,14096,a dynamic economywide model,70,,"century, and most of these losses are attribut..."
2,some more detailed work has been done at natio...,What approach did Ahmed use to estimate how ch...,14097,a modelling approach,639,,estimate how changes in climate variability m...
3,extreme sea level height fluctuations are also...,Where height fluctuations are large?,2843,extreme sea level height fluctuations are also...,0,"al. 2003 ). note that, for example, a 30 cm e...",ly coastal locations (fig. 8 left). the non-ti...
4,extreme sea level height fluctuations are also...,How non-tide sea levels are obtained?,2844,the non-tide sea levels are obtained by spectr...,167,"al. 2003 ). note that, for example, a 30 cm e...",


In [ ]:
# read in ClimateBERT answers
test_df = pd.read_parquet('test_climatebert_scored.parquet')

# evaluate ClimateBERT answers
rouge = evaluate.load('rouge')
predictions = test_df['climatebert_answer'].tolist()
references = test_df['answer'].tolist()
results = rouge.compute(predictions=predictions,
                       references=references)
print('test \n',results)

test 
 {'rouge1': np.float64(0.16085168630078878), 'rouge2': np.float64(0.1207792259959081), 'rougeL': np.float64(0.14315359243664832), 'rougeLsum': np.float64(0.14364757872822392)}


In [ ]:
# ClimateBERT tuned for QA task

# define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")
model = AutoModelForQuestionAnswering.from_pretrained("climatebert/distilroberta-base-climate-f")

## Freeze layers and only train QA head
for param in model.roberta.parameters():
    param.requires_grad = False

for param in model.qa_outputs.parameters():
    param.requires_grad = True

## Tokenize train, validation, and test sets using main tokenize function
climatebert_train = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
climatebert_valid = Dataset.from_pandas(valid_df).map(tokenize_function, batched=True)
climatebert_test = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

## Define the training arguments
climatebert_training_args = TrainingArguments(
    output_dir="./climatebert_results",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

## Prepare the Trainer
climatebert_trainer = Trainer(
    model=model,
    args=climatebert_training_args,
    train_dataset=climatebert_train,
    eval_dataset=climatebert_valid
)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/14756 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (587 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/4229 [00:00<?, ? examples/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
## Train the ClimateBERT model
climatebert_trainer.train()

## Evaluate the ClimateBERT model on the validation set
val_results = climatebert_trainer.evaluate()
print(val_results)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mjlenci (mjlenci-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,5.972100,5.697384
2,5.701800,5.624398
3,5.658400,5.607131


{'eval_loss': 5.607131004333496, 'eval_runtime': 61.7818, 'eval_samples_per_second': 68.451, 'eval_steps_per_second': 4.289, 'epoch': 3.0}


In [ ]:
## Compute ROUGE scores for ClimateBERT test

climatebert_predictions = climatebert_trainer.predict(climatebert_test)
logits = climatebert_predictions.predictions
start_logits = torch.tensor(logits[0])
end_logits = torch.tensor(logits[1])

start_positions = torch.argmax(start_logits, dim=-1)
end_positions = torch.argmax(end_logits, dim=-1)

decoded_answers = []
for i, (start, end) in enumerate(zip(start_positions, end_positions)):

    answer_tokens = climatebert_test[i]['input_ids'][start:end+1]
    decoded_answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    decoded_answers.append(decoded_answer)

references = test_df['answer'].tolist()
rouge_results = rouge.compute(predictions=decoded_answers, references=references)
print(rouge_results)

{'rouge1': np.float64(0.1246116972172919), 'rouge2': np.float64(0.07747134990986682), 'rougeL': np.float64(0.10800566734956854), 'rougeLsum': np.float64(0.10820044834220537)}


In [ ]:
climatebert_df = test_df.copy()
climatebert_df['finetuned_climatebert_answer'] = decoded_answers

climatebert_df.to_parquet('test_climatebert_scored.parquet')
climatebert_df.head()

In [ ]:


tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/14756 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/4229 [00:00<?, ? examples/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

In [ ]:
# Load the BERT base model for question answering
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")
# model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Prepare the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'eval_loss': 6.242977142333984, 'eval_model_preparation_time': 0.0051, 'eval_runtime': 128.8052, 'eval_samples_per_second': 32.833, 'eval_steps_per_second': 2.057}


In [ ]:
# Train the BERT base model
trainer.train()

# Evaluate the BERT base model on the validation set
val_results = trainer.evaluate()
print(val_results)

In [ ]:
# Predict BERT base model on test set
predictions = trainer.predict(tokenized_test)

In [ ]:
# EVERYTHING BELOW THIS IS OLD AND CAN BE REMOVED AFTER RUNNING THROUGH. LEAVING FOR NOW TO MAKE SURE WE DON'T NEED IT GOING FORWARD.

In [ ]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("bleu")
bleurt_metric = evaluate.load("bleurt")
rouge_metric = evaluate.load("rouge")


In [ ]:
# read in BERT answers
test_df = pd.read_parquet('test_bert_scored.parquet')

# evaluate BERT answers
rouge = evaluate.load('rouge')
predictions = test_df['bert_answer'].tolist()
references = test_df['answer'].tolist()
results = rouge.compute(predictions=predictions,
                       references=references)
print('test \n',results)

test 
 {'rouge1': 0.13808890794923134, 'rouge2': 0.09449441091673152, 'rougeL': 0.1206936809408421, 'rougeLsum': 0.12057938002898017}


In [ ]:
empty_answers = test_df[test_df['bert_answer'] == '']
print(empty_answers.shape)

(106, 6)


In [ ]:
def get_answer_from_model(context, question):

    inputs = tokenizer(question, context, truncation=True, return_tensors='pt')

    outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = inputs['input_ids'][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens)

    return answer

# apply on test data
test_df.loc[:, 'bert_answer'] = test_df.apply(lambda row: get_answer_from_model(row['context'], row['question']), axis=1)

# apply on subset of validation data
# valid_subset.loc[:, 'predicted_answer'] = valid_subset.apply(lambda row: get_answer_from_model(row['context'], row['question']), axis=1)